In [20]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import itertools

import healpy as hp
import pymaster as nmt

import cov_utils as cu
import cl_utils as clu
import anglib as al
import loading as load


In [4]:
def pos_def(cov):
    try:
        np.linalg.cholesky(cov)
        print('is pos def')
    except:
        print('is not positive difinite')

def is_symmetric(cov):
    test = (cov == cov.T)
    if np.where(test==False)[0].size == 0:
        print ('is symmetric')
    else :
        print('not symmetric')

def min_max(cov):
    print(cov.min(), cov.max())

In [31]:
def compute_cov(Cl, keys, keys_all, workspace, cov_workspace):
    nell = Cl['ell'].size
    ncl = len(keys)
    print(ncl*nell)
    covmat = np.zeros((ncl*nell, ncl*nell))

    for key in keys_all:
        probeA, probeB = key.split('-')
        Cl['-'.join([probeB, probeA])] = Cl[key]

    for (idx1, key1), (idx2, key2) in itertools.combinations_with_replacement(enumerate(keys), 2):
        print(key1, key2, flush=True)
        probeA, probeB = key1.split('-')
        probeC, probeD = key2.split('-')

        covmat[idx1*nell:(idx1+1)*nell, idx2*nell:(idx2+1)*nell] =\
            nmt.gaussian_covariance(cov_workspace, 0, 0, 0, 0,
                                    [Cl['-'.join([probeA, probeC])]],
                                    [Cl['-'.join([probeB, probeC])]],
                                    [Cl['-'.join([probeA, probeD])]],
                                    [Cl['-'.join([probeB, probeD])]],
                                    workspace, wb=workspace)

In [13]:
data_ref = '/home/hidra2/gouyou/euclid/nl_bias_flagship/data/'

## Get / Generate Masks

In [17]:
nside = 256

mask_fullsky = hp.read_map(data_ref+'mask/fullsky_mask_binary_NS256.fits')
fmask_fullsky = nmt.NmtField(mask_fullsky, [mask_fullsky])

mask_flagship = hp.read_map(data_ref+'mask/flagship2_mask_binary_NS256.fits')
fmask_flagship = nmt.NmtField(mask_flagship, [mask_flagship])

## Get Cl's

In [24]:
ref = '/home/hidra2/gouyou/cosmosis-standard-library/working_dir/output/fs2_nlbias/simulation/fs2_firstchain_linbias_unbinned/'
nzbins = 6

cls_gc_auto, keys_gc_auto = load.get_cosmosis_Cl(ref, nzbins, 'GC', False)
cls_gc, keys_gc = load.get_cosmosis_Cl(ref, nzbins, 'GC', True)
cls_wl, keys_wl = load.get_cosmosis_Cl(ref, nzbins, 'WL', True)
cls_ggl, keys_ggl = load.get_cosmosis_Cl(ref, nzbins, 'GGL', True)

ell_ub = cls_gc_auto['ell']
nell_ub = ell_ub.size


## Initiate binning 

In [12]:
lmin = 10
bw_lin = 50
nbl_log = 11

bnmt_lin = al.edges_binning(nside, lmin, bw_lin)
bnmt_log = al.edges_log_binning(nside, lmin, nbl_log)
print('ell_lin = {}'.format(bnmt_lin.get_effective_ells()))
print('ell_log = {}'.format(bnmt_log.get_effective_ells()))

ell_lin = [ 34.5  84.5 134.5 184.5 234.5 284.5 334.5 384.5 434.5 484.5 534.5]
ell_log = [ 11.5  17.   26.   39.5  59.   88.  131.  194.5 288.5 428. ]


## Compute workspace and mixing matrix

In [16]:
wksp_flagship_linbin = al.coupling_matrix(bnmt_lin, mask_flagship,
                    data_ref+'nmt_workspace/FS2_2pt_NmtWorkspace_NS256_LIN_LMIN10_BW50.fits')
wksp_flagship_logbin = al.coupling_matrix(bnmt_log, mask_flagship,
                    data_ref+'nmt_workspace/FS2_2pt_NmtWorkspace_NS256_LOG_LMIN10_NBL11.fits')

wksp_fullsky_linbin = al.coupling_matrix(bnmt_lin, mask_fullsky,
                    data_ref+'nmt_workspace/fullsky_2pt_NmtWorkspace_NS256_LIN_LMIN10_BW50.fits')
wksp_fullsky_logbin = al.coupling_matrix(bnmt_log, mask_fullsky,
                    data_ref+'nmt_workspace/fullsky_2pt_NmtWorkspace_NS256_LOG_LMIN10_NBL11.fits')

Compute the mixing matrix
Mixing matrix has already been calculated and is in the workspace file :  /home/hidra2/gouyou/euclid/nl_bias_flagship/data/nmt_workspace/FS2_2pt_NmtWorkspace_NS256_LIN_LMIN10_BW50.fits . Read it.
Done computing the mixing matrix. It took  0.12516379356384277 s.
Compute the mixing matrix
Mixing matrix has already been calculated and is in the workspace file :  /home/hidra2/gouyou/euclid/nl_bias_flagship/data/nmt_workspace/FS2_2pt_NmtWorkspace_NS256_LOG_LMIN10_NBL11.fits . Read it.
Done computing the mixing matrix. It took  0.08110642433166504 s.
Compute the mixing matrix
Mixing matrix has already been calculated and is in the workspace file :  /home/hidra2/gouyou/euclid/nl_bias_flagship/data/nmt_workspace/fullsky_2pt_NmtWorkspace_NS256_LIN_LMIN10_BW50.fits . Read it.
Done computing the mixing matrix. It took  0.08324861526489258 s.
Compute the mixing matrix
Mixing matrix has already been calculated and is in the workspace file :  /home/hidra2/gouyou/euclid/nl_b

In [19]:
cov_wksp_flagship_linbin = nmt.NmtCovarianceWorkspace()
cov_wksp_flagship_logbin = nmt.NmtCovarianceWorkspace()
cov_wksp_fullsky_linbin = nmt.NmtCovarianceWorkspace()
cov_wksp_fullsky_logbin = nmt.NmtCovarianceWorkspace()

cov_wksp_flagship_linbin.compute_coupling_coefficients(fmask_flagship, fmask_flagship)
cov_wksp_fullsky_linbin.compute_coupling_coefficients(fmask_fullsky, fmask_fullsky)


## Compute covariance

In [32]:
# Reference: FS2, log binning, full 3x2pt
keys_all_ref = keys_gc + keys_ggl + keys_wl
cl_ref = {}
cl_ref.update(cls_gc)
cl_ref.update(cls_wl)
cl_ref.update(cls_ggl)
cov_reference = compute_cov(cl_ref, keys_all_ref, keys_all_ref, wksp_flagship_logbin, cov_wksp_flagship_logbin)

239616


MemoryError: Unable to allocate 428. GiB for an array with shape (239616, 239616) and data type float64

### Varying binning

### Varying probes

### Varying apodization 

### With white noise Cl's